# Яндекс Музыка

В данном проекте мы сравним музыкальные предпочтения и поведение пользователей двух городов: Москвы и Санкт-Петербурга. 

Для этого мы проверим три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

Этапы проекта:
1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотез.





## Обзор данных.

Изучим данные для дальнейшего исследования.




In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('yandex_music_project.csv')
# чтение файла с данными и сохранение в df

In [4]:
df.head()
# получение первых 5 строк таблицы df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [5]:
df.info()
# получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**Выводы**

У нас есть датасет с информацией об около 65000 прослушиваний. О каждом прослушивании есть информация о названии композиции, исполнителе, жанре, а также городе, дне и времени прослушивания.




## Предобработка данных


В данных есть пропуски, названия колонок не соответствуют принятому стилю, а также есть дубликаты. В данном разделе подготовим данные для дальнейшего анализа.

In [6]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [7]:
df=df.rename(columns={'  userID':'user_id','Track':'track','  City  ':'city','Day':'day'})
#исправим названия столбцов

In [8]:
df.isna().sum()
# подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Мы переименовали столбцы и обнаружили пропуски в столбцах с названием композиции, исполнителем и жанром.

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для исследования. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам. 

In [9]:
columns_to_replace=['track','artist','genre']
for item in columns_to_replace:
    df[item]=df[item].fillna('unknown')
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

In [10]:
df.duplicated().sum()# подсчёт явных дубликатов

3826

In [11]:
df=df.drop_duplicates().reset_index(drop=True)# удаление явных дубликатов

Мы удалили явные дубликаты в датасете. Далее удалим неявные дубликаты.

In [12]:
df['genre'].sort_values().unique()
# Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [13]:
spisok_duplicates=['hip','hop','hip-hop']
df=df.replace(spisok_duplicates,'hiphop')
# Устранение неявных дубликатов

В данном разделе мы исправили названия столбцов, пропущенне значения заменили на 'unknown', удалили явные и неявные дубликаты.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице (в данных есть информация только об этих днях). Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [14]:
grouped=df.groupby('city').count()
grouped['user_id']
# Подсчёт прослушиваний в каждом городе

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

In [15]:
grouped=df.groupby('day').count()
grouped['user_id']
# Подсчёт прослушиваний в каждый из трёх дней

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Рассмотрим каждый город в отдельности.

Создадим функцию для посчета прослушиваний в разных городах и в разные дни недели.

In [16]:
# <создание функции number_tracks()>
def number_tracks(day,city):
    df1=df[df['day']==day]
    track_list=df1[df1['city']==city]
    track_list_count=track_list.count()['user_id']
    return track_list_count
    

# Функция для подсчёта прослушиваний для конкретного города и дня.


In [17]:
number_tracks('Monday','Moscow')

# количество прослушиваний в Москве по понедельникам

15740

In [18]:
number_tracks('Monday','Saint-Petersburg')

# количество прослушиваний в Москве по понедельникам

5614

In [19]:
number_tracks('Wednesday','Moscow')

# количество прослушиваний в Москве по понедельникам

11056

In [20]:
number_tracks('Wednesday','Saint-Petersburg')

# количество прослушиваний в Москве по понедельникам

7003

In [21]:
number_tracks('Friday','Moscow')

# количество прослушиваний в Москве по понедельникам

15945

In [22]:
number_tracks('Friday','Saint-Petersburg')

# количество прослушиваний в Москве по понедельникам

5895

In [23]:
columns=['city', 'monday', 'wednesday', 'friday']

data = pd.DataFrame(data=[['Moscow',15740,11056,15945], ['Saint-Petrsburg',5614,7003,5895]], columns=columns)
data
# Таблица с результатами

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petrsburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.


Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [24]:
moscow_general=df[df['city']=='Moscow']
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'


In [25]:
spb_general=df[df['city']=='Saint-Petersburg']
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'




Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция вернет информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [26]:
def genre_weekday(table,day,time1,time2):
    genre_df=table[table['day']==day]
    genre_df=genre_df[genre_df['time']>time1]
    genre_df=genre_df[genre_df['time']<time2]
    genre_df_count=genre_df.groupby('genre')['genre'].count()
    genre_df_sorted=genre_df_count.sort_values(ascending=False)
    return genre_df_sorted[:10]







Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [27]:
genre_weekday(moscow_general,'Monday','07:00','11:00')
# вызов функции для утра понедельника в Москве (вместо df — таблица spb_general)

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [28]:
genre_weekday(spb_general,'Monday','07:00','11:00')

# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [29]:
genre_weekday(moscow_general,'Friday','17:00','23:00')

# вызов функции для вечера пятницы в Москве

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [30]:
genre_weekday(spb_general,'Friday','17:00','23:00')
# вызов функции для вечера пятницы в Петербурге

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

Сгруппируем таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [31]:
moscow_genres=moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)


In [32]:
moscow_genres.head(10)# просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Повторим то же и для Петербурга.


In [33]:
spb_genres=spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)


In [34]:
spb_genres.head(10)# просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

Это был учебный проект, поэтому мы не проводили проверок **статистических гипотез**. Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
